In [1]:
import sqlite3  
  
con = sqlite3.connect("employee.db")  
print("Database opened successfully")  
  
con.execute("create table Employees (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT NOT NULL, roll TEXT UNIQUE NOT NULL, email TEXT NOT NULL, marks_short TEXT NOT NULL, marks_des TEXT NOT NULL)")  
  
print("Table created successfully")  
  
con.close()

Database opened successfully


OperationalError: table Employees already exists

In [ ]:
from werkzeug.wrappers import Request, Response
from flask import *
import sqlite3

app = Flask(__name__)

@app.route("/")
def index():
    return render_template("A_index.html")

@app.route("/add")
def add():
    return render_template("A_add.html")

@app.route("/savedetails", methods=["POST", "GET"])
def saveDetails():
    msg = "msg"
    if request.method == "POST":
        try:
            name = request.form["name"]
            email = request.form["email"]
            address = request.form["address"]
            with sqlite3.connect("employee.db") as con:
                cur = con.cursor()
                cur.execute("INSERT into Employees (name, email, address) values (?,?,?)", (name, email, address))
                con.commit()
                msg = "Employee successfully Added"
        except:
            con.rollback()
            msg = "We can not add the employee to the list"
        finally:
            return render_template("A_success.html", msg=msg)

@app.route("/view")
def view():
    con = sqlite3.connect("employee.db")
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    cur.execute("select * from Employees")
    rows = cur.fetchall()
    return render_template("A_view.html", rows=rows)

@app.route("/delete")
def delete():
    return render_template("A_delete.html")

@app.route("/deleterecord", methods=["POST"])
def deleterecord():
    id = request.form["id"]

    # Check if ID is a number (integer)
    try:
        id = int(id)
    except ValueError:
        msg = "Invalid ID. Please enter a number."
        return render_template("A_delete_record.html", msg=msg)

    with sqlite3.connect("employee.db") as con:
        try:
            cur = con.cursor()

            # Check if ID exists before deletion
            cur.execute("SELECT count(*) FROM Employees WHERE id = ?", (id,))
            count = cur.fetchone()[0]  # Get the count from the first column

            if count > 0:
                cur.execute("delete from Employees where id = ?", (id,))
                con.commit()
                msg = "Record successfully deleted"
            else:
                msg = "Record with ID {} does not exist.".format(id)

        except:
            msg = "Record could not be deleted."
        finally:
            return render_template("A_delete_record.html", msg=msg)

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 8090, app)


 * Running on http://localhost:8090/ (Press CTRL+C to quit)
